# Conditional MC of Heston Model based on QE Scheme

In [1]:
import numpy as np
import heston_cmc_qe as heston
import time
import pyfeng as pf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# parameters
strike = [100.0, 140.0, 70.0]
forward = 100
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]

case = np.zeros([3, 6])
case[0] = [1,   0.5, -0.9, 10, 0.04, np.sqrt(0.04)]
case[1] = [0.9, 0.3, -0.5, 15, 0.04, np.sqrt(0.04)]
case[2] = [1,   1,   -0.3, 5,  0.09, np.sqrt(0.09)]

In [3]:
# compute cmc price
for i in range(3):
    vov, kappa, rho, texp, theta, sigma = case[i]
    start = time.time()

    heston_cmc_qe = heston.HestonCondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike)])
    for d in range(len(delta)):
        price_cmc[d, :] = heston_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e5, seed=123456)

    end = time.time()
    
    print('Case %s:' % i)
    print(price_cmc)
    print('Running time is %.3f seconds.' % (end - start) + '\n')

Case 0:
[[14.52722285  0.19584722 37.20591415]
 [13.56691261  0.26568546 36.12295964]
 [13.22061601  0.29003533 35.9154245 ]
 [13.12057087  0.29501411 35.90207168]
 [13.1042753   0.29476289 35.89245755]
 [13.09047939  0.29547721 35.86410028]]
Running time is 89.549 seconds.

Case 1:
[[16.14957874  4.76025232 37.46778665]
 [16.4168519   5.03174927 37.2821383 ]
 [16.61786281  5.14344251 37.21595949]
 [16.64846896  5.14132547 37.1891449 ]
 [16.67691989  5.15529418 37.20344323]
 [16.64978732  5.12726313 37.19663617]]
Running time is 127.015 seconds.

Case 2:
[[21.29889948  9.2869257  39.01120605]
 [21.56671782  9.67348515 38.91635189]
 [21.74480387  9.92863213 38.78834587]
 [21.77517348  9.96452557 38.76137082]
 [21.77743678  9.96544321 38.761411  ]
 [21.80425894  9.99245464 38.7805373 ]]
Running time is 53.132 seconds.



In [4]:
# compute std error of cmc price
n = 50
for i in range(3):
    start = time.time()
    vov, kappa, rho, texp, theta, sigma = case[i]

    heston_cmc_qe = heston.HestonCondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike), n])
    for j in tqdm(range(n)):
        for d in range(len(delta)):
            price_cmc[d, :, j] = heston_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e4)
            
    end = time.time()
    print('Case %s:' % i)
    print(price_cmc.mean(axis=2))
    print(price_cmc.std(axis=2))
    print('Running time is %.3f seconds.' % (end - start) + '\n')

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [07:32<00:00,  9.05s/it]


Case 0:
[[14.5135643   0.1954222  37.1528721 ]
 [13.56681958  0.26565958 36.12768962]
 [13.19108236  0.29160396 35.89108888]
 [13.09872786  0.29709118 35.86159984]
 [13.08706455  0.29633349 35.85839164]
 [13.07866733  0.29629266 35.84403563]]
[[0.09495364 0.00229899 0.20477555]
 [0.07700054 0.00264051 0.16221602]
 [0.05822169 0.00294066 0.12239383]
 [0.08040474 0.00254826 0.18517111]
 [0.07475837 0.00195271 0.17059226]
 [0.05701166 0.00231027 0.1285361 ]]
Running time is 452.365 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [10:41<00:00, 12.83s/it]


Case 1:
[[16.14192096  4.7514775  37.46072346]
 [16.45593409  5.0775051  37.28008256]
 [16.61702747  5.15131144 37.18511373]
 [16.6582092   5.1503099  37.19142045]
 [16.65465842  5.14115995 37.17518654]
 [16.65479299  5.14129617 37.17754509]]
[[0.04803649 0.05745457 0.08332121]
 [0.04493472 0.04683139 0.07851349]
 [0.0431723  0.05895522 0.07449869]
 [0.04083082 0.04699761 0.0949146 ]
 [0.0440387  0.05308105 0.07595908]
 [0.04022257 0.04836162 0.07868294]]
Running time is 641.474 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [04:45<00:00,  5.71s/it]


Case 2:
[[21.2987581   9.27205634 39.04076979]
 [21.55464073  9.6517802  38.92587232]
 [21.7392762   9.91333709 38.80129458]
 [21.78737344  9.97040333 38.78179056]
 [21.79715154  9.98780613 38.76928165]
 [21.80361507  9.99057713 38.77792258]]
[[0.0311757  0.05136215 0.06375496]
 [0.03364137 0.0503933  0.07890605]
 [0.03695861 0.04668313 0.06982219]
 [0.03424185 0.03938702 0.06880689]
 [0.02800778 0.04839543 0.06456386]
 [0.03462467 0.04446041 0.06241863]]
Running time is 285.624 seconds.

